<a href="https://colab.research.google.com/github/sixth-group/project2/blob/main/VisionWithTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Download train data

In [ ]:
# !gdown 1-7ljOXuzdIeXTCRuxZFGH2Sg3IanH-WJ
# !gdown 1-5xCKnZ7N7X6m0pBV1yZDoTmJjAQpttF
# !unzip -q train_data.zip
# !unzip -q test_data.zip

In [7]:
!gdown 1-7ljOXuzdIeXTCRuxZFGH2Sg3IanH-WJ
# https://drive.google.com/file/d/1-7ljOXuzdIeXTCRuxZFGH2Sg3IanH-WJ/view?usp=sharing
!unzip train_data.zip
%rm train_data.zip

Streaming output truncated to the last 5000 lines.
  inflating: train_data/10/fac187c69fc74a48d28e626bd17fe5b0d36db1e6_1618836534.jpg  
  inflating: train_data/10/9a9f9ff4d3dcfc88e6b9209178d4ee791142fdaa_1618725283.jpg  
  inflating: train_data/10/958281d27ab722113c92180afe17db62bfb6a5d0_1623743398.jpg  
  inflating: train_data/10/8a94fada61bcb6dda8983dc36ddf2ec5d32cbf9f_1629199040.jpg  
  inflating: train_data/10/d72c1b799a927514b1fd81d9ee93208734be4d61_1613975034.jpg  
  inflating: train_data/10/6550a6492a93a9542b7757c898369f7996bb62b7_1613978514.jpg  
  inflating: train_data/10/3465c07dc8a3521f0303cec03ebbc4ac2d738317_1630174913.jpg  
  inflating: train_data/10/74a657eb2f8bc1a9bcd42fee32d04b1a400fcddf_1627746721.jpg  
  inflating: train_data/10/387c4a5ef5d37c263ebe457a2aab08890da843ca_1620800188.jpg  
  inflating: train_data/10/01330eb299562e0e10bdc560e7ed12754f49703b_1621081068.jpg  
  inflating: train_data/10/1ea75672aea3496a099cc3c8d8e2c01306456a52_1619644153.jpg  
  inflating: t

In [5]:
rm -rf train_data/

In [9]:
len(train_data)

NameError: ignored

In [ ]:
# !pip install timm

In [ ]:
import torch
from torch import nn
from torch.optim import Adam
from torchvision.transforms import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split

from torchvision.models import resnet50

# Set up the directory paths
train_data_dir = '/content/train_data'
input_size = (224, 224)

# Define data transformations
train_transform = transforms.Compose([
    transforms.RandomVerticalFlip(),
    transforms.RandomHorizontalFlip(),
    transforms.RandomAffine(degrees=20, translate=(0.2, 0.2)),
    transforms.Resize(input_size),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# Create datasets
train_dataset = ImageFolder(train_data_dir, transform=train_transform)
dataset_size = len(train_dataset)

# Define the size of your training set, for example, 80% of the dataset
train_size = int(0.8 * dataset_size)

# The rest of the data will be used for validation
valid_size = dataset_size - train_size

# Split your dataset
train_dataset, valid_dataset = random_split(train_dataset, [train_size, valid_size])

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=128, shuffle=False)

# Load the pre-trained ResNet50 model
base_model = resnet50(pretrained=True)

# Replace the top layer of the pre-trained model
num_classes = 10
base_model.fc = nn.Linear(base_model.fc.in_features, num_classes)

# Define the device (CPU or GPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Move the model to the device
model = base_model.to(device)

# Define the optimizer and loss function
optimizer = Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    train_correct = 0
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        train_loss += loss.item() * images.size(0)
        train_correct += (predicted == labels).sum().item()

    train_loss /= len(train_dataset)
    train_acc = train_correct / len(train_dataset)

    model.eval()
    valid_loss = 0.0
    valid_correct = 0
    with torch.no_grad():
        for images, labels in valid_loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            loss = criterion(outputs, labels)

            valid_loss += loss.item() * images.size(0)
            valid_correct += (predicted == labels).sum().item()

    valid_loss /= len(valid_dataset)
    valid_acc = valid_correct / len(valid_dataset)

    print(f"Epoch {epoch+1}/{num_epochs}: "
          f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, "
          f"Valid Loss: {valid_loss:.4f}, Valid Acc: {valid_acc:.4f}")

Epoch 1/10: Train Loss: 2.0628, Train Acc: 0.2694, Valid Loss: 2.0774, Valid Acc: 0.2903
Epoch 2/10: Train Loss: 1.8364, Train Acc: 0.3587, Valid Loss: 2.5663, Valid Acc: 0.2467
Epoch 3/10: Train Loss: 1.7268, Train Acc: 0.3948, Valid Loss: 2.0828, Valid Acc: 0.2913


In [ ]:
import torch
from torch import nn
from torch.optim import Adam
from torchvision.transforms import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

from torchvision.models import densenet121

# # Set up the directory paths
# train_data_dir = '/content/train_data'
# input_size = (224, 224)

# # Define data transformations
# train_transform = transforms.Compose([
#     transforms.RandomVerticalFlip(),
#     transforms.RandomHorizontalFlip(),
#     transforms.RandomAffine(degrees=20, translate=(0.2, 0.2)),
#     transforms.Resize(input_size),
#     transforms.ToTensor()
# ])

# valid_transform = transforms.Compose([
#     transforms.Resize(input_size),
#     transforms.ToTensor()
# ])

# # Create datasets
# train_dataset = ImageFolder(train_data_dir, transform=train_transform)
# valid_dataset = ImageFolder(train_data_dir, transform=valid_transform)

# # Create data loaders
# train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
# valid_loader = DataLoader(valid_dataset, batch_size=64, shuffle=False)

# Set up the directory paths
train_data_dir = '/content/train_data'
input_size = (224, 224)

# Define data transformations
train_transform = transforms.Compose([
    transforms.RandomVerticalFlip(),
    transforms.RandomHorizontalFlip(),
    transforms.RandomAffine(degrees=20, translate=(0.2, 0.2)),
    transforms.Resize(input_size),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# Create datasets
train_dataset = ImageFolder(train_data_dir, transform=train_transform)
dataset_size = len(train_dataset)

# Define the size of your training set, for example, 80% of the dataset
train_size = int(0.8 * dataset_size)

# The rest of the data will be used for validation
valid_size = dataset_size - train_size

# Split your dataset
train_dataset, valid_dataset = random_split(train_dataset, [train_size, valid_size])

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=128, shuffle=False)

# Load the pre-trained DenseNet121 model
base_model = densenet121(pretrained=True)

# Replace the top layer of the pre-trained model
num_classes = len(train_dataset.classes)
base_model.classifier = nn.Linear(base_model.classifier.in_features, num_classes)

# Define the device (CPU or GPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Move the model to the device
model = base_model.to(device)

# Define the optimizer and loss function
optimizer = Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    train_correct = 0
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        train_loss += loss.item() * images.size(0)
        train_correct += (predicted == labels).sum().item()

    train_loss /= len(train_dataset)
    train_acc = train_correct / len(train_dataset)

    model.eval()
    valid_loss = 0.0
    valid_correct = 0
    with torch.no_grad():
        for images, labels in valid_loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            loss = criterion(outputs, labels)

            valid_loss += loss.item() * images.size(0)
            valid_correct += (predicted == labels).sum().item()

    valid_loss /= len(valid_dataset)
    valid_acc = valid_correct / len(valid_dataset)

    print(f"Epoch {epoch+1}/{num_epochs}: "
          f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, "
          f"Valid Loss: {valid_loss:.4f}, Valid Acc: {valid_acc:.4f}")

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.cache/torch/hub/checkpoints/densenet121-a639ec97.pth
100%|██████████| 30.8M/30.8M [00:00<00:00, 92.3MB/s]


In [ ]:
!gdown 1-7aMdKW4KcCKLwoUKC3XxdIwfIKkzwx6
!unzip test_data.zip
%rm test_data.zip

In [ ]:
# rm -rf test_data/